In [8]:
import os
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# import statsmodels.api as sm
import pycaret.clustering as pycc
import pycaret.utils as pycu

In [2]:
load_dotenv()

True

In [3]:
df = pd.read_csv(os.getenv("TRANSFORM_DATA_PATH"), sep=";")
df.head(5)

,Nombre,Temporada,Equipo,Partidos,Minutos,Puntos,T2 Anotados,T2 Lanzados,% T2,T3 Anotados,...,Rebotes,Asistencias,Robos,Perdidas,Tapones,Tapones Recibidos,Mates,Faltas Cometidas,Faltas Recibidas,Valoracion
0,"PARRA, FAUSTINE LAURE CLEMENCE",2021,SPAR GIRONA,3.0,9.97,3.33,0.67,1.00,0.67,0.67,...,1.67,0.67,0.67,1.00,0.00,0.00,0.0,1.33,0.00,2.67
1,"NAVARRO LLORCA, CANDIDA",2009,HONDARRIBIA - IRÚN,19.0,9.96,2.32,0.95,2.21,0.43,0.00,...,1.42,0.37,0.11,0.47,0.11,0.00,0.0,1.32,0.63,1.68
2,"CORRALES BOHOYO, ELENA",2017,NISSAN AL-QÁZERES EXTREMADURA,26.0,9.95,1.46,0.35,1.15,0.30,0.15,...,0.50,0.46,0.38,0.69,0.00,0.12,0.0,0.81,0.35,0.38
3,"HERNANDEZ ORTIZ, ZOE",2019,MANN-FILTER CASABLANCA,19.0,9.95,2.79,0.32,0.63,0.50,0.68,...,1.11,0.37,0.47,0.47,0.05,0.11,0.0,0.95,0.26,2.68
4,"MILOGLAV, CARMEN",2016,PERFUMERIAS AVENIDA,11.0,9.94,1.27,0.36,0.55,0.67,0.18,...,0.91,0.55,0.64,1.73,0.09,0.00,0.0,1.55,0.18,-1.18


In [9]:
s = pycc.setup(df, 
          ignore_features=['Nombre', 'Temporada', 'Equipo', 'Minutos'],
          normalize=True,
          session_id=123
        )

,Description,Value
0,Session id,123
1,Original data shape,"(5445, 27)"
2,Transformed data shape,"(5445, 23)"
3,Ignore features,4
4,Numeric features,23
5,Preprocess,True
6,Imputation type,simple
7,Numeric imputation,mean
8,Categorical imputation,mode
9,Normalize,True


In [15]:
catalog = pycc.models()

In [16]:
results = []

for m in catalog.index:
    # Entrenamos con 4 clusters
    mdl   = pycc.create_model(m, num_clusters=4, verbose=False)
    grid  = pycc.pull()                      # última salida de create_model
    grid['Model_ID'] = m                # para saber a qué modelo pertenece
    results.append(grid)

In [22]:
scores = pd.concat(results, ignore_index=True).sort_values('Silhouette', ascending=False)
scores

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness,Model_ID
3,0.6195,17.8869,0.3424,0,0,0,sc
0,0.2190,2027.9884,1.4786,0,0,0,kmeans
2,0.1989,75.2807,1.3186,0,0,0,meanshift
7,0.1946,1716.0641,1.6789,0,0,0,birch
4,0.1840,1747.5114,1.7409,0,0,0,hclust
5,0.0698,127.4786,0.7300,0,0,0,dbscan
1,0.0483,76.7332,1.5912,0,0,0,ap
6,-0.3308,24.2652,0.9269,0,0,0,optics


In [27]:
best = scores.iloc[0]['Model_ID']
best_model   = pycc.create_model('kmeans', num_clusters=4)
clustered_df = pycc.assign_model(best_model)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.2190,2027.9884,1.4786,0,0,0


In [28]:
pycc.plot_model(best_model, plot='cluster')